## Importando Bibliotecas e Carregando Bases de Dados

In [11]:
import pandas as pd
import numpy as np
import os
import re
import time
import unicodedata
from collections import defaultdict

from itertools import combinations
from ortools.sat.python import cp_model

In [12]:
participants = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Participants')
must_together = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Must_Together_2')
must_avoid = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Must_Avoid')
ramo_alias = pd.read_csv('ramo_alias.csv')
affinity_overrides = pd.read_csv('affinity_overrides.csv')

display(participants.head())
display(must_together.head())
display(must_avoid.head())
display(ramo_alias.head())
display(affinity_overrides.head())

,ID,Nome_Pessoa,Nome_Empresa,Ramo,Pref1_Ramo,Pref2_Ramo,Pref3_Ramo,Fixo,Mesa_Fixa,Assento_Fixo
0,0,Adriana de Souza Torres,Dri Torres Terapeuta,Terapia,NaN,NaN,NaN,False,NaN,NaN
1,1,Ailton Nunes,Bela Festa Locações,Eventos,NaN,NaN,NaN,False,NaN,NaN
2,2,Alessandra Ribeiro,Remax B12,Construção e Imobiliário,NaN,NaN,NaN,False,NaN,NaN
3,3,Alessandra Sallum Bueno,EXPERIÊNCIA 40+,Eventos,NaN,NaN,NaN,False,NaN,NaN
4,4,Ana Claudia Soares,Terapeuta,Terapia,NaN,NaN,NaN,False,NaN,NaN


,ID_A,Nome_A,ID_B,Nome_B
0,55,Silene Darin,57,Silviä Castro
1,62,Walter Junior,11,Caroline Scorvo
2,41,Mariana Gobbo,26,Fabiano Souza Ramos
3,41,Mariana Gobbo,51,Rosana Matos
4,41,Mariana Gobbo,4,Ana Claudia Soares


,ID_A,Nome_A,ID_B,Nome_B
0,14,Cidinha Lins,7,Bervelyn Alecrim
1,14,Cidinha Lins,54,Silene Silva
2,18,Deise Gressens,58,Tiago Vasconcelos
3,22,Evelyn Silva,60,Tiago Santos
4,40,Maria Vasconcelos,58,Tiago Vasconcelos


,entrada,normalizado,macro_ramo
0,Adestrador,pet,pet
1,Automação,tecnologia,tecnologia
2,Coaching,consultoria,consultoria
3,Construção e Imobiliário,construcao_imobiliario,construcao_imobiliario
4,Consultoria Comercial,consultoria,consultoria


,ramo_a,ramo_b,sim
0,marketing,eventos,0.90
1,marketing,varejo_especializado,0.85
2,marketing,automotivo,0.80
3,marketing,tecnologia,0.80
4,marketing,saude,0.75


## Script Otimização

In [ ]:
# === Otimizador de Rodada de Negócios (CP-SAT / OR-Tools) — versão com fixos robustos ===
# Entradas já carregadas: participants, must_together, must_avoid, ramo_alias, affinity_overrides

# -------------------------
# Parâmetros do evento
# -------------------------
ROUNDS = 4
TABLES = 17
SEATS_PER_TABLE = 4

# -------------------------
# Pesos (ajuste fino)
# -------------------------
W_TOGETHER = 5000
W_SIM = 10
W_PREF1 = 300
W_PREF2 = 180
W_PREF3 = 90
PREF_FILLED_BOOST = 1.5
W_OVER = 800
DEFAULT_SIM_SAME = 0.30
DEFAULT_SIM_DIFF = 0.55

MAX_TIME_S = 30
N_WORKERS = 8

# -------------------------
# Normalização por alias
# -------------------------
def strip_accents(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s

alias_map = {}
if {'entrada','normalizado'}.issubset({c.lower() for c in ramo_alias.columns}):
    _entrada = [c for c in ramo_alias.columns if c.lower()=='entrada'][0]
    _normalizado = [c for c in ramo_alias.columns if c.lower()=='normalizado'][0]
    for _,row in ramo_alias.iterrows():
        k = strip_accents(row[_entrada])
        v = str(row[_normalizado]).strip().lower()
        if k:
            alias_map[k] = v

def alias_norm(x: str) -> str:
    k = strip_accents(x)
    return alias_map.get(k, k if k else "outros")

# -------------------------
# Preparo dos participants
# -------------------------
cols = {c.lower(): c for c in participants.columns}
getcol = lambda name: cols.get(name)

id_col       = getcol('id')
nome_col     = getcol('nome_pessoa') or getcol('nome') or getcol('pessoa') or getcol('nome completo')
empresa_col  = getcol('nome_empresa') or getcol('empresa')
ramo_col     = getcol('ramo')
pref1_col    = getcol('pref1_ramo')
pref2_col    = getcol('pref2_ramo')
pref3_col    = getcol('pref3_ramo')
fixo_col     = getcol('fixo')          # pode não existir
mesa_fixa_col= getcol('mesa_fixa') or getcol('mesa fixa') or getcol('mesa fixa ')

dfP = participants.copy()

# id
if id_col is None:
    dfP['id'] = np.arange(len(dfP))
    id_col = 'id'

# nome/empresa/ramo (obrigatórios)
if nome_col is None:
    raise ValueError("Coluna 'nome_pessoa' (ou equivalente) não encontrada em Participants.")
if empresa_col is None:
    raise ValueError("Coluna 'nome_empresa' (ou equivalente) não encontrada em Participants.")
if ramo_col is None:
    raise ValueError("Coluna 'ramo' não encontrada em Participants.")

# normaliza ramo e prefs
dfP['ramo_norm'] = dfP[ramo_col].astype(str).apply(alias_norm)
for colname, newname in [(pref1_col,'pref1_norm'), (pref2_col,'pref2_norm'), (pref3_col,'pref3_norm')]:
    if colname:
        dfP[newname] = dfP[colname].astype(str).apply(lambda x: alias_norm(x) if str(x).strip() != "" else "")
    else:
        dfP[newname] = ""

# mesa fixa: força fixo=1 quando houver mesa definida
if mesa_fixa_col is None:
    dfP['mesa_fixa'] = np.nan
    mesa_fixa_col = 'mesa_fixa'
else:
    dfP['mesa_fixa'] = pd.to_numeric(dfP[mesa_fixa_col], errors='coerce')

# Heurística de base 1 → base 0 (se todos valores válidos estiverem em 1..TABLES e não houver 0)
if dfP['mesa_fixa'].notna().any():
    mf_nonnull = dfP['mesa_fixa'].dropna()
    cond_one_based = (mf_nonnull.min() >= 1) and (mf_nonnull.max() <= TABLES) and (0 not in set(mf_nonnull.astype(int)))
    if cond_one_based:
        dfP.loc[dfP['mesa_fixa'].notna(), 'mesa_fixa'] = dfP.loc[dfP['mesa_fixa'].notna(), 'mesa_fixa'].astype(int) - 1

# fixo (coluna pode não existir; aceita números, bools, strings)
def _to_bool01(x):
    if pd.isna(x): return 0
    if isinstance(x, (int, np.integer)): return 1 if int(x)!=0 else 0
    if isinstance(x, float): 
        if np.isnan(x): return 0
        return 1 if int(round(x))!=0 else 0
    s = str(x).strip().lower()
    return 1 if s in {'1','true','t','sim','yes','y'} else 0

if fixo_col is None:
    dfP['fixo'] = 0
else:
    dfP['fixo'] = dfP[fixo_col].apply(_to_bool01)

# **Regra nova**: quem tem mesa_fixa preenchida vira fixo=1 (sobrepõe)
dfP.loc[dfP['mesa_fixa'].notna(), 'fixo'] = 1

# sanity básico
N = len(dfP)
if N != TABLES * SEATS_PER_TABLE:
    print(f"[Aviso] Há {N} participantes, mas a configuração espera {TABLES*SEATS_PER_TABLE}. "
          f"Ainda assim o solver tentará alocar exatamente 4 por mesa por rodada.")

# mapeamentos
P_ids = list(dfP[id_col].tolist())
id_to_idx = {pid:i for i,pid in enumerate(P_ids)}

names    = dfP[nome_col].astype(str).tolist()
empresas = dfP[empresa_col].astype(str).tolist()
ind_of   = dfP['ramo_norm'].astype(str).tolist()
pref1    = dfP['pref1_norm'].astype(str).tolist()
pref2    = dfP['pref2_norm'].astype(str).tolist()
pref3    = dfP['pref3_norm'].astype(str).tolist()
has_pref = [(str(pref1[i]).strip()!="") or (str(pref2[i]).strip()!="") or (str(pref3[i]).strip()!="") for i in range(N)]

fixed_info = {}
for i in range(N):
    if dfP['fixo'].iloc[i] == 1 and pd.notna(dfP['mesa_fixa'].iloc[i]):
        mf = int(dfP['mesa_fixa'].iloc[i])
        if not (0 <= mf < TABLES):
            raise ValueError(f"Mesa fixa fora do range [0..{TABLES-1}] para {names[i]}: {mf}")
        fixed_info[i] = mf  # fixo por mesa

# -------------------------
# must_together / must_avoid -> índices
# -------------------------
def get_pair_df(df, ca='id_a', cb='id_b'):
    if df is None or df.empty:
        return []
    cols = {c.lower(): c for c in df.columns}
    A = cols.get(ca.lower(), list(df.columns)[0])
    B = cols.get(cb.lower(), list(df.columns)[1])
    out = []
    for _,row in df.iterrows():
        a = row[A]; b = row[B]
        if pd.isna(a) or pd.isna(b): continue
        a = id_to_idx.get(a, None); b = id_to_idx.get(b, None)
        if a is None or b is None or a == b: continue
        if a > b: a,b = b,a
        out.append((a,b))
    return sorted(set(out))

must_together_pairs = get_pair_df(must_together)
must_avoid_pairs    = get_pair_df(must_avoid)

# -------------------------
# Pré-checagens (fail-fast)
# -------------------------
# a) capacidade por mesa só com fixos
from collections import Counter
fix_count = Counter(fixed_info.values())
overcap = {t:c for t,c in fix_count.items() if c > SEATS_PER_TABLE}
if overcap:
    raise RuntimeError(f"Capacidade excedida com fixos: {overcap}")

# b) must_avoid com dois fixos mesma mesa
bad_avoid = []
for (a,b) in must_avoid_pairs:
    if (a in fixed_info) and (b in fixed_info) and (fixed_info[a]==fixed_info[b]):
        bad_avoid.append((P_ids[a], P_ids[b], fixed_info[a]))
if bad_avoid:
    raise RuntimeError(f"Par(es) must_avoid estão fixos na mesma mesa (inviável): {bad_avoid}")

# c) must_together com dois fixos em mesas diferentes
bad_mt = []
for (a,b) in must_together_pairs:
    if (a in fixed_info) and (b in fixed_info) and (fixed_info[a]!=fixed_info[b]):
        bad_mt.append((P_ids[a], P_ids[b], fixed_info[a], fixed_info[b]))
if bad_mt:
    raise RuntimeError(f"Par(es) must_together fixos em mesas distintas (inviável): {bad_mt}")

# -------------------------
# Afinidade (sim) por ramo
# -------------------------
def sim_lookup(ind_a, ind_b):
    ia = str(ind_a).strip().lower()
    ib = str(ind_b).strip().lower()
    if len(affinity_overrides) > 0:
        cols = {c.lower(): c for c in affinity_overrides.columns}
        ra = cols.get('ramo_a', list(affinity_overrides.columns)[0])
        rb = cols.get('ramo_b', list(affinity_overrides.columns)[1])
        rs = cols.get('sim',    list(affinity_overrides.columns)[2])
        mask = ((affinity_overrides[ra].str.strip().str.lower()==ia) & 
                (affinity_overrides[rb].str.strip().str.lower()==ib))
        val = affinity_overrides.loc[mask, rs]
        if len(val)==0:
            mask = ((affinity_overrides[ra].str.strip().str.lower()==ib) & 
                    (affinity_overrides[rb].str.strip().str.lower()==ia))
            val = affinity_overrides.loc[mask, rs]
        if len(val)>0:
            try:
                return float(val.iloc[0])
            except:
                pass
    return DEFAULT_SIM_SAME if ia == ib else DEFAULT_SIM_DIFF

sim100 = {}
inds_unique = sorted(set(ind_of))
for a in inds_unique:
    for b in inds_unique:
        sim100[(a,b)] = int(round(sim_lookup(a,b) * 100))

# -------------------------
# Score de preferência (direcional)
# -------------------------
def pref_dir_score(i_from, i_to):
    ramo_to = ind_of[i_to]
    s = 0.0
    if ramo_to and ramo_to == pref1[i_from]: s += W_PREF1
    if ramo_to and ramo_to == pref2[i_from]: s += W_PREF2
    if ramo_to and ramo_to == pref3[i_from]: s += W_PREF3
    if has_pref[i_from]:
        s *= PREF_FILLED_BOOST
    return int(round(s))

# -------------------------
# Modelo CP-SAT
# -------------------------
R = range(ROUNDS)
T = range(TABLES)
P = range(N)
model = cp_model.CpModel()

# X[i,r,t] = 1 se pessoa i está na mesa t na rodada r
X = {(i,r,t): model.NewBoolVar(f"X_{i}_{r}_{t}") for i in P for r in R for t in T}

# 1) Cada pessoa ocupa exatamente 1 mesa por rodada
for i in P:
    for r in R:
        model.Add(sum(X[i,r,t] for t in T) == 1)

# 2) Capacidade: 4 por mesa/rodada
for r in R:
    for t in T:
        model.Add(sum(X[i,r,t] for i in P) == SEATS_PER_TABLE)

# 3) Fixos: mesa fixa em todas as rodadas
for i, mf in fixed_info.items():
    for r in R:
        model.Add(X[i,r,mf] == 1)

# 3b) Não fixos não podem repetir mesa (cada mesa no máx. 1x para a pessoa)
for i in P:
    if i not in fixed_info:
        for t in T:
            model.Add(sum(X[i,r,t] for r in R) <= 1)

# 4) must_avoid
for (a,b) in must_avoid_pairs:
    for r in R:
        for t in T:
            model.Add(X[a,r,t] + X[b,r,t] <= 1)

# 5) Indicadores de encontro por par/rodada/mesa (somente a<b)
pairs = [(a,b) for (a,b) in combinations(P,2)]
B = {}
for (a,b) in pairs:
    for r in R:
        for t in T:
            B[a,b,r,t] = model.NewBoolVar(f"B_{a}_{b}_{r}_{t}")
            model.AddBoolAnd([X[a,r,t], X[b,r,t]]).OnlyEnforceIf(B[a,b,r,t])
            model.AddBoolOr([X[a,r,t].Not(), X[b,r,t].Not()]).OnlyEnforceIf(B[a,b,r,t].Not())

# 6) Pares não must_together: no máximo 1 encontro no evento
must_together_set = set(must_together_pairs)
for (a,b) in pairs:
    if (a,b) not in must_together_set:
        model.Add(sum(B[a,b,r,t] for r in R for t in T) <= 1)

# 7) Y[a,b] para must_together atendido
Y = {}
for (a,b) in must_together_pairs:
    Y[a,b] = model.NewBoolVar(f"Y_{a}_{b}")
    model.Add(Y[a,b] <= sum(B[a,b,r,t] for r in R for t in T))
    for r in R:
        for t in T:
            model.Add(Y[a,b] >= B[a,b,r,t])

# 8) Diversidade por mesa: penaliza cnt(ramo) > 1
inds_present = sorted(set(ind_of))
over_vars = []
for r in R:
    for t in T:
        for ind in inds_present:
            cnt = model.NewIntVar(0, SEATS_PER_TABLE, f"cnt_{ind}_r{r}_t{t}")
            inds_idx = [i for i in P if ind_of[i] == ind]
            if len(inds_idx) == 0:
                model.Add(cnt == 0)
            else:
                model.Add(cnt == sum(X[i,r,t] for i in inds_idx))
            over = model.NewIntVar(0, SEATS_PER_TABLE, f"over_{ind}_r{r}_t{t}")
            model.Add(over >= cnt - 1)
            model.Add(over >= 0)
            over_vars.append(over)

# -------------------------
# Função objetivo (minimização)
# -------------------------
obj_terms = []

# must_together (max)
for (a,b), yvar in Y.items():
    obj_terms.append((-W_TOGETHER, yvar))

# afinidade + preferências (max)
for (a,b) in pairs:
    s_sim  = W_SIM * sim100[(ind_of[a], ind_of[b])]
    s_pref = pref_dir_score(a, b) + pref_dir_score(b, a)
    s_pair = s_sim + s_pref
    if s_pair != 0:
        for r in R:
            for t in T:
                obj_terms.append((-s_pair, B[a,b,r,t]))

# diversidade (min)
for over in over_vars:
    obj_terms.append((W_OVER, over))

model.Minimize(sum(coef * var for (coef, var) in obj_terms))

# -------------------------
# Solver
# -------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = MAX_TIME_S
solver.parameters.num_search_workers = N_WORKERS

status = solver.Solve(model)
if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    raise RuntimeError("Modelo infeasible (verifique pares must_avoid/must_together e distribuição de fixos).")

# -------------------------
# Extrai solução
# -------------------------
assign = {(r,t): [] for r in R for t in T}
for i in range(N):
    for r in R:
        for t in T:
            if solver.Value(X[i,r,t]) == 1:
                assign[(r,t)].append(i)

# DataFrame longo
rows = []
for r in R:
    for t in T:
        for i in assign[(r,t)]:
            rows.append({
                "rodada": r,
                "mesa": t,
                "id": P_ids[i],
                "nome_pessoa": names[i],
                "nome_empresa": empresas[i],
                "ramo_norm": ind_of[i],
                "pref1_norm": pref1[i],
                "pref2_norm": pref2[i],
                "pref3_norm": pref3[i],
                "fixo": int(dfP['fixo'].iloc[i]),
                "mesa_fixa": int(dfP['mesa_fixa'].iloc[i]) if pd.notna(dfP['mesa_fixa'].iloc[i]) else np.nan,
            })
schedule_df = pd.DataFrame(rows).sort_values(["rodada","mesa","nome_pessoa"]).reset_index(drop=True)

# Visual compacto
def to_wide(g):
    ps = list(g.sort_values("nome_pessoa")["nome_pessoa"])
    row = {"rodada": int(g["rodada"].iloc[0]), "mesa": int(g["mesa"].iloc[0])}
    for k, nm in enumerate(ps[:SEATS_PER_TABLE], start=1):
        row[f"p{k}"] = nm
    return pd.Series(row)

schedule_wide = (
    schedule_df.groupby(["rodada","mesa"], as_index=False)
    .apply(to_wide)
    .sort_values(["rodada","mesa"])
    .reset_index(drop=True)
)

# Relatórios rápidos
mt_att = []
for (a,b) in must_together_pairs:
    atendeu = 0; r_hit = None; t_hit = None
    for r in R:
        for t in T:
            if solver.Value(B[a,b,r,t]) == 1:
                atendeu = 1; r_hit = r; t_hit = t; break
        if atendeu: break
    mt_att.append({"id_a": P_ids[a], "nome_a": names[a], "id_b": P_ids[b], "nome_b": names[b],
                   "atendido": atendeu, "rodada": r_hit, "mesa": t_hit})
must_together_report = pd.DataFrame(mt_att)

ma_viol = []
for (a,b) in must_avoid_pairs:
    for r in R:
        for t in T:
            if solver.Value(B[a,b,r,t]) == 1:
                ma_viol.append({"id_a": P_ids[a], "nome_a": names[a], "id_b": P_ids[b], "nome_b": names[b],
                                "rodada": r, "mesa": t})
must_avoid_violations = pd.DataFrame(ma_viol)

# Checagem extra: fixos realmente permaneceram na mesma mesa (e na mesa certa)
fixed_check = []
for i, mf in fixed_info.items():
    mesas_i = sorted(schedule_df.query("id == @P_ids[i]")["mesa"].unique().tolist())
    fixed_check.append({
        "id": P_ids[i], "nome": names[i], "mesa_fixa_esperada": mf, "mesas_encontradas": mesas_i,
        "ok": (mesas_i == [mf])
    })
fixed_check_df = pd.DataFrame(fixed_check)

print("Resumo:")
print("- must_together atendidos:", must_together_report['atendido'].sum(), "/", len(must_together_report))
print("- must_avoid violações   :", len(must_avoid_violations))
if not fixed_check_df.empty and not fixed_check_df['ok'].all():
    print("[ATENÇÃO] Algum fixo não ficou 100% na mesa esperada. Veja fixed_check_df.")

RuntimeError: Modelo infeasible (verifique pares must_avoid/must_together e distribuição de fixos).

## Validações Gerais

In [ ]:
# Validando encontros duplicados

schedule_wide[
    (schedule_wide['p1'] == sc hedule_wide['p2']) |
    (schedule_wide['p1'] == schedule_wide['p3']) |
    (schedule_wide['p1'] == schedule_wide['p4']) |
    (schedule_wide['p2'] == schedule_wide['p3']) |
    (schedule_wide['p2'] == schedule_wide['p4']) |
    (schedule_wide['p3'] == schedule_wide['p4'])
]

NameError: name 'schedule_wide' is not defined

In [ ]:
# =========================
# Validações avançadas / KPIs
# Pré: schedule_df, participants, must_together, must_avoid, affinity_overrides
# =========================
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict

# --------- helpers ----------
def _col(df, *cands):
    m = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in m:
            return m[c.lower()]
    return None

pid = _col(schedule_df, 'id')
rod = _col(schedule_df, 'rodada')
tab = _col(schedule_df, 'mesa')
ram = _col(schedule_df, 'ramo_norm') or _col(schedule_df,'ramo')
name = _col(schedule_df, 'nome_pessoa') or _col(schedule_df,'nome')
emp  = _col(schedule_df, 'nome_empresa') or _col(schedule_df,'empresa')

if not all([pid, rod, tab, ram]):
    raise ValueError("schedule_df precisa conter: id, rodada, mesa, ramo_norm/ramo.")

# matriz de similaridade (0..1) baseada em affinity_overrides + defaults
DEFAULT_SIM_SAME = 0.30
DEFAULT_SIM_DIFF = 0.55
cols = {c.lower(): c for c in affinity_overrides.columns}
ra = cols.get('ramo_a', list(affinity_overrides.columns)[0])
rb = cols.get('ramo_b', list(affinity_overrides.columns)[1])
rs = cols.get('sim',    list(affinity_overrides.columns)[2])

def sim_lookup(a, b):
    a = str(a).strip().lower()
    b = str(b).strip().lower()
    mask = (affinity_overrides[ra].str.strip().str.lower().eq(a) &
            affinity_overrides[rb].str.strip().str.lower().eq(b))
    v = affinity_overrides.loc[mask, rs]
    if not v.empty:
        return float(v.iloc[0])
    mask = (affinity_overrides[ra].str.strip().str.lower().eq(b) &
            affinity_overrides[rb].str.strip().str.lower().eq(a))
    v = affinity_overrides.loc[mask, rs]
    if not v.empty:
        return float(v.iloc[0])
    return DEFAULT_SIM_SAME if a == b else DEFAULT_SIM_DIFF

# ---------- 1) Diversidade e afinidade por mesa ----------
def sim_pairwise_mean(g):
    # g: DF de uma mesa em uma rodada
    inds = g[ram].tolist()
    if len(inds) < 2:
        return np.nan
    sims = []
    for a,b in combinations(inds,2):
        sims.append(sim_lookup(a,b))
    return np.mean(sims) if sims else np.nan

def simpson_diversity(g):
    # 1 - sum(p_i^2), quanto mais perto de 1, mais diverso (com 4 pessoas, max=0.75 quando 4 ramos distintos)
    counts = g[ram].value_counts(normalize=True)
    return float(1 - np.sum(counts**2))

by_table = (schedule_df
            .groupby([rod, tab])
            .apply(lambda g: pd.Series({
                "n_pessoas": len(g),
                "ramos_unicos": g[ram].nunique(),
                "simpson_div": simpson_diversity(g),
                "sim_pair_mean": sim_pairwise_mean(g)
            }))
            .reset_index()
            .sort_values([rod, tab]))

# ---------- 2) KPI por rodada: distribuição de diversidade ----------
kpi_round = (by_table
             .assign(flag4=lambda d: (d['ramos_unicos']==4).astype(int),
                     flag3=lambda d: (d['ramos_unicos']==3).astype(int),
                     flag2=lambda d: (d['ramos_unicos']==2).astype(int),
                     flag1=lambda d: (d['ramos_unicos']==1).astype(int))
             .groupby(rod)
             .agg(
                 mesas=('mesa','count'),
                 pct_4_ramos=('flag4', lambda x: round(100*x.mean(),2)),
                 pct_3_ramos=('flag3', lambda x: round(100*x.mean(),2)),
                 pct_2_ramos=('flag2', lambda x: round(100*x.mean(),2)),
                 pct_1_ramo =('flag1', lambda x: round(100*x.mean(),2)),
                 afinidade_media=('sim_pair_mean','mean'),
                 simpson_medio =('simpson_div','mean')
             )
             .reset_index())

print("=== KPI por rodada (diversidade & afinidade) ===")
display(kpi_round)

# ---------- 3) Parceiros por pessoa (únicos, repetidos, mesmo ramo) ----------
# lista de parceiros por (rodada, mesa)
pairs_rt = schedule_df.groupby([rod, tab]).apply(lambda g: list(combinations(sorted(g[pid]),2))).explode()
pairs_rt = pairs_rt.dropna().reset_index(name='pair')
pairs_rt[['id_a','id_b']] = pd.DataFrame(pairs_rt['pair'].tolist(), index=pairs_rt.index)
pairs_rt = pairs_rt.drop(columns=['pair'])

# parceiros únicos por pessoa
partners = defaultdict(set)
partners_same_industry = defaultdict(int)
id_to_ind = dict(zip(schedule_df[pid], schedule_df[ram]))
for _, row in pairs_rt.iterrows():
    a,b = int(row['id_a']), int(row['id_b'])
    partners[a].add(b); partners[b].add(a)
    if id_to_ind.get(a) == id_to_ind.get(b):
        partners_same_industry[a] += 1
        partners_same_industry[b] += 1

df_partners = pd.DataFrame({
    "id": list(schedule_df[pid].unique())
})
df_partners['n_parceiros_unicos'] = df_partners['id'].map(lambda x: len(partners.get(x,set())))
df_partners['reps_mesmo_ramo']   = df_partners['id'].map(lambda x: partners_same_industry.get(x,0))
# máximo teórico de parceiros únicos em 4 rodadas com mesas de 4 = 12
df_partners['max_teorico'] = 12
df_partners['gap_parceiros'] = df_partners['max_teorico'] - df_partners['n_parceiros_unicos']

print("=== Pessoas com menos parceiros únicos (top 15) ===")
display(df_partners.sort_values(['n_parceiros_unicos','reps_mesmo_ramo'], ascending=[True, True]).head(15))

print("=== Pessoas que mais sentaram com mesmo ramo (top 15) ===")
display(df_partners.sort_values('reps_mesmo_ramo', ascending=False).head(15))

# ---------- 4) Repetição de parceiro (não must_together) > 1 ----------
# conta encontros por par
enc_by_pair = (pairs_rt.groupby(['id_a','id_b'])
               .size().reset_index(name='encontros'))
# normaliza ordem (id_a < id_b)
enc_by_pair['a'] = enc_by_pair[['id_a','id_b']].min(axis=1)
enc_by_pair['b'] = enc_by_pair[['id_a','id_b']].max(axis=1)
enc_by_pair = (enc_by_pair.groupby(['a','b'])['encontros'].sum()
               .reset_index().rename(columns={'a':'id_a','b':'id_b'}))

# remove must_together da checagem
mt_cols = {c.lower(): c for c in must_together.columns}
mta = mt_cols.get('id_a', list(must_together.columns)[0])
mtb = mt_cols.get('id_b', list(must_together.columns)[1])
mt_set = set(tuple(sorted((int(r[mta]), int(r[mtb])))) for _,r in must_together.iterrows())

rep_nao_mt = enc_by_pair[(enc_by_pair['encontros'] > 1) &
                         (~enc_by_pair.apply(lambda r: (r['id_a'], r['id_b']) in mt_set, axis=1))]
print("=== Pares repetidos (>1) que não são must_together ===")
display(rep_nao_mt.sort_values('encontros', ascending=False))

# ---------- 5) Repetição de MESA por pessoa (ideal: 4 mesas distintas) ----------
mesas_por_pessoa = (schedule_df.groupby([pid])[tab].nunique()
                    .reset_index().rename(columns={tab:'mesas_distintas'}))
mesas_por_pessoa['ideal'] = 4
mesas_por_pessoa['gap'] = mesas_por_pessoa['ideal'] - mesas_por_pessoa['mesas_distintas']
rep_mesa = mesas_por_pessoa[mesas_por_pessoa['mesas_distintas'] < 4].sort_values('mesas_distintas')
print("=== Pessoas que repetiram mesa entre rodadas ===")
display(rep_mesa.head(20))

# ---------- 6) Mesas críticas (baixa diversidade / alta homogeneidade) ----------
criticas = by_table.sort_values(['ramos_unicos','simpson_div','sim_pair_mean']).query("n_pessoas==4").head(20)
print("=== Mesas mais críticas (ordenadas por baixa diversidade) ===")
display(criticas)

# ---------- 7) Afinidade média por mesa (útil p/ ver “quão diversa” ficou sob sua métrica) ----------
print("=== Top 20 mesas com MAIOR afinidade média (complementaridades fortes) ===")
display(by_table.sort_values('sim_pair_mean', ascending=False).head(20))

print("=== Top 20 mesas com MENOR afinidade média (potencial ajuste) ===")
display(by_table.sort_values('sim_pair_mean', ascending=True).head(20))

# ---------- 8) Preferências: cobertura por pessoa (teve pelo menos 1 encontro compatível ao longo do evento?) ----------
# reconstruir prefs da base participantes (caso schedule_df não tenha)
p_id = _col(participants, 'id')
pf1  = _col(participants, 'pref1_norm') or _col(participants, 'pref1_ramo')
pf2  = _col(participants, 'pref2_norm') or _col(participants, 'pref2_ramo')
pf3  = _col(participants, 'pref3_norm') or _col(participants, 'pref3_ramo')
rP   = _col(participants, 'ramo_norm') or _col(participants, 'ramo')

tmpP = participants[[p_id, rP, pf1, pf2, pf3]].copy()
tmpP.columns = ['id','ramo_norm','pref1','pref2','pref3']
schedP = schedule_df.merge(tmpP, on='id', how='left', suffixes=('','_p'))

# checa, para cada pessoa, em cada rodada, se teve ao menos 1 parceiro do ramo em suas prefs
def _prefs_set(row):
    s=set()
    for c in ['pref1','pref2','pref3']:
        v = str(row[c]).strip().lower()
        if v and v!='nan': s.add(v)
    return s

schedP['_prefset'] = schedP.apply(_prefs_set, axis=1)
has_pref = schedP['_prefset'].apply(lambda s: len(s)>0)

pref_hits = []
for (r,t), g in schedP.groupby([rod, tab]):
    ids = list(g['id'])
    ramo_map = dict(zip(g['id'], g[ram]))
    pref_map = dict(zip(g['id'], g['_prefset']))
    for i in ids:
        if len(pref_map[i])==0: 
            continue
        others = [j for j in ids if j != i]
        ok = any(ramo_map[j] in pref_map[i] for j in others)
        pref_hits.append((i,r,ok))

pref_df = pd.DataFrame(pref_hits, columns=['id','rodada','hit'])
# cobertura ao longo do evento (>=1 rodada)
cov_evento = (pref_df.groupby('id')['hit'].max().reset_index()
              .merge(tmpP[['id']], on='id', how='right'))
cov_evento['has_pref'] = participants[p_id].map(lambda x: len(_prefs_set(tmpP[tmpP['id']==x].iloc[0]))>0)
cov_evento = cov_evento[cov_evento['has_pref']==True]
cov_evento['hit'] = cov_evento['hit'].fillna(False)

print("=== Cobertura de preferências ao longo do evento ===")
total_com_pref = cov_evento.shape[0]
atendidos = int(cov_evento['hit'].sum())
print(f"- Pessoas com preferência: {total_com_pref}")
print(f"- Atendidas >=1x:         {atendidos}  ({round(100*atendidos/max(1,total_com_pref),2)}%)")

nao_atendidos = cov_evento[cov_evento['hit']==False]['id'].tolist()
df_nao_atendidos = schedule_df[schedule_df['id'].isin(nao_atendidos)][['id', name, emp]].drop_duplicates()
print("=== Pessoas com pref. que NÃO tiveram encontro compatível (listar p/ ajustes finos) ===")
display(df_nao_atendidos)


In [ ]:
backtesting_path = fr'C:\Users\felip\Documents\FELIPE\Netweaving_Conecta\backtesting'
os.chdir(backtesting_path)

# pasta de saída (mude se quiser)
stamp = time.strftime("%Y%m%d_%H%M%S")
out_dir = f"exports_{stamp}"
os.makedirs(out_dir, exist_ok=True)

def _exists(name):
    return name in globals() and isinstance(globals()[name], pd.DataFrame)

def _save(name, filename=None):
    if not _exists(name):
        return None
    df = globals()[name]
    fn = filename or f"{name}.csv"
    path = os.path.join(out_dir, fn)
    try:
        df.to_csv(path, index=False, encoding="utf-8-sig")
        return {"name": name, "file": fn, "rows": len(df), "cols": list(df.columns)}
    except Exception as e:
        return {"name": name, "file": fn, "error": str(e)}

# lista de possíveis DFs a exportar (adicione/remova se quiser)
candidates = [
    # principais
    "schedule_df", "schedule_wide",
    # validações básicas
    "viol_once_per_round", "viol_four_total", "viol_capacity", "missing_tables_df",
    "fixed_mismatch", "must_avoid_violations", "must_together_unmet",
    "repeat_pairs_df", "diversity_viol",
    # preferências (do primeiro pacote)
    "pref_global", "pref_by_round", "pref_unserved", "top_pairs",
    # KPIs/validações avançadas
    "by_table", "kpi_round", "df_partners", "rep_nao_mt", "rep_mesa", "criticas"
]

manifest = []
for name in candidates:
    rec = _save(name)
    if rec: manifest.append(rec)

# também salva um manifesto .csv e imprime um resumo
manifest_df = pd.DataFrame(manifest)
manifest_path = os.path.join(out_dir, "_manifest.csv")
manifest_df.to_csv(manifest_path, index=False, encoding="utf-8-sig")

print(f"Arquivos salvos em: {out_dir}")
if not manifest_df.empty:
    display(manifest_df[["name","file","rows"]])
else:
    print("Nenhum DataFrame esperado foi encontrado no ambiente.")


## Validações Individuais

In [ ]:
# Verificando se os que deveriam ser fixos realmente ficaram nas mesmas mesas
nomes = [
    "Caroline Scorvo",
    "Cleiton Vicente",
    "Fabiano Souza Ramos",
    "Márcia Fonseca Borges",
    "Nanci Toledo",
    "Tiago Oliveira"
]

for nome in nomes:
    temp = schedule_wide[
        (schedule_wide['p1'] == nome) | (schedule_wide['p2'] == nome) | (schedule_wide['p3'] == nome) | (schedule_wide['p4'] == nome)
    ]
    unicos = temp['mesa'].unique()
    n_unicos = temp['mesa'].nunique()
    print(f"{nome} - Mesas únicas: {n_unicos} - Mesas: {list(unicos)}")

In [ ]:
# Teste Individual
nome = "Deise Gressens"

schedule_wide[
        (schedule_wide['p1'] == nome) | (schedule_wide['p2'] == nome) | (schedule_wide['p3'] == nome) | (schedule_wide['p4'] == nome)
]

In [ ]:
# Primeira rodada Cidinha

**Potenciais Melhorias**
- Primeira Rodada Cidinha

## Formatação Desejada Tabela Final

In [ ]:
os.chdir(fr"C:\Users\felip\Documents\FELIPE\Netweaving_Conecta")
schedule_wide.to_csv('schedule_wide.csv', index=False, encoding='utf-8-sig')

In [ ]:
schedule_wide

In [ ]:
participants

In [ ]:
# Gera df_final_premium (fixos) e df_final_standard (móveis)
# Fonte da verdade para FIXO = schedule_wide (mesma mesa nas 4 rodadas)
# Usa participants só para preencher EMPRESA. Não depende de Fixo/Mesa_Fixa do participants.

import pandas as pd
import numpy as np
import unicodedata

# aceita tanto schedule_wide quanto schedue_wide (typo)
if 'schedule_wide' not in globals() and 'schedue_wide' in globals():
    schedule_wide = schedue_wide

def _strip(s):
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKD", str(s).strip())
    return "".join(c for c in s if not unicodedata.combining(c))

def _det(df, *names):
    m = {c.lower(): c for c in df.columns}
    for n in names:
        if n and n.lower() in m:
            return m[n.lower()]
    return None

# 1) schedule_wide -> long
id_cols   = [c for c in schedule_wide.columns if c.lower() in {"rodada","mesa"}]
seat_cols = [c for c in schedule_wide.columns if c.lower().startswith("p")]
sw_long = (
    schedule_wide.melt(id_vars=id_cols, value_vars=seat_cols,
                       var_name="seat", value_name="nome_pessoa")
    .dropna(subset=["nome_pessoa"])
    .copy()
)
sw_long["mesa_label"]   = (sw_long["mesa"].astype(int) + 1).map(lambda x: f"MESA {x}")
sw_long["rodada_label"] = (sw_long["rodada"].astype(int) + 1).map(lambda x: f"{x}ª RODADA")

# 2) FIXO pelo schedule: quem ficou na MESMA mesa em todas as rodadas
mesas_dist = sw_long.groupby("nome_pessoa")["mesa"].nunique()
fixed_by_schedule = mesas_dist.eq(1).astype(int).rename("FIXO").reset_index()

# mesa fixa (rótulo) derivada do schedule
mesa_fixa_num = (
    sw_long.groupby("nome_pessoa")["mesa"]
           .agg(lambda s: s.iloc[0] if s.nunique()==1 else np.nan)
           .rename("MESA_FIXA_NUM")
           .reset_index()
)
mesa_fixa_num["MESA_FIXA"] = mesa_fixa_num["MESA_FIXA_NUM"].apply(
    lambda v: f"MESA {int(v)+1}" if pd.notna(v) else np.nan
)

# 3) EMPRESA via participants (sem afetar FIXO)
c_nome    = _det(participants, "Nome_Pessoa","nome_pessoa","Nome","Pessoa","nome")
c_empresa = _det(participants, "Nome_Empresa","nome_empresa","Empresa","empresa")

parts = participants.copy()
parts["_nome_key"] = parts[c_nome].map(lambda s: _strip(s).lower())
emp_por_nome = (
    parts.groupby("_nome_key")[c_empresa]
         .apply(lambda s: next((x for x in s if pd.notna(x) and str(x).strip()!=''), ""))
         .rename("EMPRESA")
         .reset_index()
)

sw_long["_nome_key"] = sw_long["nome_pessoa"].map(lambda s: _strip(s).lower())

# 4) base por pessoa (merge correto pela _nome_key)
pessoas = (sw_long[["nome_pessoa","_nome_key"]].drop_duplicates()
           .merge(fixed_by_schedule, on="nome_pessoa", how="left")
           .merge(mesa_fixa_num, on="nome_pessoa", how="left")
           .merge(emp_por_nome, on="_nome_key", how="left")
           .drop(columns=["_nome_key"])
           .rename(columns={"nome_pessoa":"NOME"}))
pessoas["EMPRESA"] = pessoas["EMPRESA"].fillna("").astype(str)

# 5) rodada × mesa por pessoa
rodada_mesa = (sw_long
    .pivot_table(index="nome_pessoa", columns="rodada_label",
                 values="mesa_label", aggfunc="first")
    .reset_index()
    .rename(columns={"nome_pessoa":"NOME"})
)

# 6) consolidação e split
pivot = pessoas.merge(rodada_mesa, on="NOME", how="left")
rod_cols = [c for c in ["1ª RODADA","2ª RODADA","3ª RODADA","4ª RODADA"] if c in pivot.columns]
pivot = pivot[["NOME","EMPRESA","FIXO","MESA_FIXA"] + rod_cols]

df_final_premium  = pivot[pivot["FIXO"]==1].copy()
df_final_standard = pivot[pivot["FIXO"]==0].copy()

# ordenação
if not df_final_premium.empty:
    ord_ = df_final_premium["MESA_FIXA"].str.extract(r"(\d+)")[0].astype(float)
    df_final_premium = (df_final_premium
                        .assign(_ord=ord_).sort_values(["_ord","NOME"])
                        .drop(columns=["_ord"]))
df_final_standard = df_final_standard.sort_values("NOME")
df_final_standard = df_final_standard.drop(columns=["FIXO","MESA_FIXA"])

In [ ]:
final_path = fr'C:\Users\felip\Documents\FELIPE\Netweaving_Conecta\saida_final'
os.chdir(final_path)

df_final_standard.to_excel('df_final_standard.xlsx', index=False)
df_final_premium.to_excel('df_final_premium.xlsx', index=False)